# Pushup form checker whoooo

Hierin staat alles wat je nodig hebt om een ongeveer correcte push-up checker te hebben!
Er zal nog veel getweaked moeten worden om het accurater te krijgen, en eventueel andere oefeningen toe te voegen.


# 0. Installs =)

In [1]:
# Uncomment if needed to install packages
%pip install ultralytics opencv-python numpy pandas scikit-learn matplotlib pyyaml tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1. Imports & config setup
Import alle benodigheden en defineert de paths naar data

Checked ook voor mijn best weights bestand wat later van pas komt.

In [2]:

from ultralytics import YOLO
import cv2, os, math, time, random
import numpy as np
import pandas as pd
from pathlib import Path
from collections import deque
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt


DATASET_DIR = "C:/Data/hboict/Sem7-AIFS/Personal_Project"
RUNS_DIR = "runs"
RUN_NAME = "pushup-cls42"  
BEST_WEIGHTS = f"{RUNS_DIR}/{RUN_NAME}/weights/best.pt"  


OUTPUT_FRAMES_DIR = "data/frames"  
IMGSZ = 224

print("BEST_WEIGHTS:", BEST_WEIGHTS)
print("Exists?", Path(BEST_WEIGHTS).resolve(), Path(BEST_WEIGHTS).exists())


BEST_WEIGHTS: runs/pushup-cls42/weights/best.pt
Exists? C:\Data\hboict\Sem7-AIFS\Personal_Project\runs\pushup-cls42\weights\best.pt True


# 2. Random checks voor data hoeveelheid

In [3]:

def count_images(root):
    root = Path(root)
    rows = []
    for split in ["train","val","test"]:
        for cls in ["correct","incorrect"]:
            n = len(list((root/split/cls).glob("*.jpg")))
            rows.append({"split":split,"class":cls,"count":n})
    return pd.DataFrame(rows)

df_counts = count_images(OUTPUT_FRAMES_DIR)
try:
    from caas_jupyter_tools import display_dataframe_to_user
    display_dataframe_to_user("Dataset image counts", df_counts)
except Exception:
    print(df_counts)

print("\nIf any class has 0 images in *train*, fix your splits before training.\n")


   split      class  count
0  train    correct    295
1  train  incorrect    326
2    val    correct     38
3    val  incorrect    139
4   test    correct     66
5   test  incorrect     47

If any class has 0 images in *train*, fix your splits before training.



# 3. Classifier training
Training YOLO on my defined image set/ how long, how fast etc.

Uncomment als weer wilt trainen

In [7]:

from ultralytics import YOLO
model = YOLO('yolov8n-pose.pt')  
model.train(
    data=OUTPUT_FRAMES_DIR,  
    epochs=20,               # training time
    imgsz=IMGSZ,             # input image size
    project=RUNS_DIR,        
    name=RUN_NAME,           # run name
    lr0=0.00001,             # learning rate
    batch=64,                # batch sizes
    augment=True,            # data augmentation
    patience=20              # early stopping
)



New https://pypi.org/project/ultralytics/8.3.223 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.203  Python-3.13.7 torch-2.8.0+cpu CPU (13th Gen Intel Core i7-13700H)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/frames, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=1e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pushup-cls426, nbs=64, nms=False, opset=None, 

RuntimeError: Dataset 'data/frames' error  [Errno 13] Permission denied: 'data/frames'

# 4. Evaluatie classifier + confusion matrix generation
Na het trainen een korte evaluation met een confusion matrix generation.

In [ ]:

best_weights = Path(BEST_WEIGHTS)
if not best_weights.exists():
    raise FileNotFoundError(f"Best weights not found at {best_weights}. Train first or fix path.")

eval_model = YOLO(str(best_weights))
print("Class mapping from model:", eval_model.names)

metrics = eval_model.val(data=str(OUTPUT_FRAMES_DIR), imgsz=IMGSZ, split="test")
print(metrics)

# Confusion matrix and classification report
class_names = [eval_model.names[i] for i in sorted(eval_model.names.keys())]
name_to_index = {v:k for k,v in eval_model.names.items()}

y_true, y_pred = [], []
test_dir = Path(OUTPUT_FRAMES_DIR) / "test"
img_paths = []
for cls in class_names:
    cls_idx = name_to_index[cls]
    for p in (test_dir / cls).glob("*.jpg"):
        img_paths.append((str(p), cls_idx))

bs = 64
for i in range(0, len(img_paths), bs):
    batch = img_paths[i:i+bs]
    imgs = [b[0] for b in batch]
    gts = [b[1] for b in batch]
    preds = eval_model(imgs, verbose=False)
    for gt, pred in zip(gts, preds):
        pred_cls = int(np.argmax(pred.probs.data))
        y_true.append(gt)
        y_pred.append(pred_cls)

cm = confusion_matrix(y_true, y_pred, labels=[name_to_index[c] for c in class_names])
report = classification_report(y_true, y_pred, target_names=class_names, digits=4)
print(report)

def plot_confusion_matrix(cm, class_names, out_path: Path):
    fig = plt.figure(figsize=(5,4))
    plt.imshow(cm, interpolation='nearest')
    plt.title('Confusion matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    cm_normalized = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-9)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, f"{cm[i, j]}\n({cm_normalized[i, j]:.2f})",
                     horizontalalignment="center", verticalalignment="center")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    out_path.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_path, dpi=200)
    plt.close(fig)

plot_confusion_matrix(cm, class_names, Path(RUNS_DIR) / RUN_NAME / "confusion_matrix.png")
print("Saved confusion matrix to:", Path(RUNS_DIR) / RUN_NAME / "confusion_matrix.png")


Class mapping from model: {0: 'correct', 1: 'incorrect'}
Ultralytics 8.3.203  Python-3.13.7 torch-2.8.0+cpu CPU (13th Gen Intel Core i7-13700H)


YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: C:\Data\hboict\Sem7-AIFS\Personal_Project\data\frames\train... found 621 images in 2 classes  
val: C:\Data\hboict\Sem7-AIFS\Personal_Project\data\frames\val... found 177 images in 2 classes  
test: C:\Data\hboict\Sem7-AIFS\Personal_Project\data\frames\test... found 113 images in 2 classes  
test: Fast image access  (ping: 0.00.0 ms, read: 31.45.0 MB/s, size: 14.3 KB)
test: Scanning C:\Data\hboict\Sem7-AIFS\Personal_Project\data\frames\test... 113 images, 0 corrupt: 100% ━━━━━━━━━━━━ 113/113 131.1Kit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 8/8 7.4it/s 1.1s0.2s
                   all      0.867          1
Speed: 0.0ms preprocess, 6.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to C:\Data\hboict\Sem7-AIFS\Personal_Project\runs\classify\val5
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.

# 5. Geometrie zooi en pose assesments

In [ ]:

def angle(a, b, c):
    # angle at b: a-b-c
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosang = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    cosang = np.clip(cosang, -1.0, 1.0)
    return math.degrees(math.acos(cosang))

def line_point_distance(p, a, b):
    # distance from point p to line ab
    p, a, b = np.array(p), np.array(a), np.array(b)
    if np.allclose(a,b):
        return np.linalg.norm(p-a)
    return np.linalg.norm(np.cross(b-a, a-p)) / (np.linalg.norm(b-a) + 1e-6)

# keypoint indices (YOLOv8-pose)
KS = dict(nose=0, ls=5, rs=6, le=7, re=8, lw=9, rw=10, lh=11, rh=12, lk=13, rk=14, la=15, ra=16)

def get_xy(kps, i):
    # kps shape: (17,3) -> x,y,conf
    return (float(kps[i,0]), float(kps[i,1]))

def valid_triplet(kps, a,b,c, minconf=0.35):
    return kps[a,2]>minconf and kps[b,2]>minconf and kps[c,2]>minconf

def assess_form(kps, phase="bottom"):
    issues = []
    # Depth
    left_elbow = angle(get_xy(kps, KS['ls']), get_xy(kps, KS['le']), get_xy(kps, KS['lw'])) if valid_triplet(kps, KS['ls'], KS['le'], KS['lw']) else None
    right_elbow = angle(get_xy(kps, KS['rs']), get_xy(kps, KS['re']), get_xy(kps, KS['rw'])) if valid_triplet(kps, KS['rs'], KS['re'], KS['rw']) else None
    elbow_min = None
    if left_elbow is not None and right_elbow is not None:
        elbow_min = min(left_elbow, right_elbow)
    elif left_elbow is not None:
        elbow_min = left_elbow
    elif right_elbow is not None:
        elbow_min = right_elbow
    # Depth check only at bottom phase
    if phase == "bottom" and elbow_min is not None and elbow_min > 120:
        issues.append("Depth too shallow (elbows not bent enough)")

    # Hip sag / pike
    side = "r" if kps[KS['rs'],2] > kps[KS['ls'],2] else "l"
    parts = ['rs','rh','ra'] if side == 'r' else ['ls','lh','la']
    if kps[KS[parts[0]],2]>0.35 and kps[KS[parts[1]],2]>0.35 and kps[KS[parts[2]],2]>0.35:
        shoulder = get_xy(kps, KS[parts[0]])
        hip = get_xy(kps, KS[parts[1]])
        ankle = get_xy(kps, KS[parts[2]])
        sag = line_point_distance(hip, shoulder, ankle)
        norm = np.linalg.norm(np.array(shoulder) - np.array(ankle)) + 1e-6
        if sag / norm > 0.12:  # relaxed
            issues.append("Hips sagging (keep core tight)")

    # Elbow flaring (only at bottom)
    if kps[KS['rs'],2]>0.35 and kps[KS['rh'],2]>0.35 and kps[KS['re'],2]>0.35:
        rs, rh, re = get_xy(kps, KS['rs']), get_xy(kps, KS['rh']), get_xy(kps, KS['re'])
        torso_vec = np.array(rh) - np.array(rs)
        arm_vec = np.array(re) - np.array(rs)
        cosang = abs(np.dot(torso_vec, arm_vec)) / (np.linalg.norm(torso_vec)*np.linalg.norm(arm_vec)+1e-6)
        flare_angle = math.degrees(math.acos(np.clip(cosang, -1, 1)))
        if flare_angle > 70:  # slightly relaxed position instead of 45°
            issues.append("Elbows flaring out (keep ~45° from torso)")

    return issues


# 6. Pose-based rep count


In [ ]:

class PoseRepCounter:
    def __init__(self, pose_model, eval_model, frames_for_classify=7, vote_strategy="majority"):
        self.model = pose_model
        self.eval_model = eval_model
        self.prev_y = deque(maxlen=5)
        self.state = "up"
        self.last_bottom_time = None
        self.reps = 0
        self.rep_events = []  # list of dicts with rep info
        self.frames_buffer = deque(maxlen=frames_for_classify)
        self.frames_for_classify = frames_for_classify
        self.vote_strategy = vote_strategy

    def classify_buffer(self):
        if len(self.frames_buffer) == 0:
            return None, None
        preds = self.eval_model(list(self.frames_buffer), verbose=False)
        top1 = [int(p.probs.top1) for p in preds]
        if self.vote_strategy == "majority":
            final_pred = max(set(top1), key=top1.count)
        else:
            final_pred = top1[-1]
        label = self.eval_model.names[final_pred]
        confs = [float(p.probs.top1conf) for p in preds]
        return label, float(np.mean(confs))

    def update(self, frame):
        self.frames_buffer.append(frame.copy())
        res = self.model(frame, verbose=False)[0]
        kps_best = None

        if res.keypoints is not None and len(res.keypoints) > 0:
            # Select best keypoint set (highest avg confidence)
            best_i = 0
            best_score = -1
            for i, kp in enumerate(res.keypoints):
                conf = float(kp.conf.mean())
                if conf > best_score:
                    best_score = conf
                    best_i = i
            kps = res.keypoints[best_i].data[0].cpu().numpy()  # (17,3)
            kps_best = kps

            # Compute mean y of shoulders and hips
            y_points = []
            for idx in [KS['ls'], KS['rs'], KS['lh'], KS['rh']]:
                if kps[idx,2] > 0.35:
                    y_points.append(kps[idx,1])
            if len(y_points)>=2:
                y_mean = float(np.mean(y_points))
                self.prev_y.append(y_mean)

            if len(self.prev_y) == self.prev_y.maxlen:
                dy = self.prev_y[-1] - self.prev_y[0]
                # simple hysteresis on y to detect down/up
                if self.state == "up" and dy > 8:
                    self.state = "down"
                elif self.state == "down" and dy < -8:
                    self.state = "up"
                    # Completed a rep
                    self.reps += 1
                    # Classify the buffered frames around bottom position
                    feedback_label, avg_conf = self.classify_buffer()
                    issues = assess_form(kps, phase="bottom")
                    event = {
                        "rep": self.reps,
                        "time": time.time(),
                        "feedback": feedback_label,
                        "issues": issues,
                        "kps_conf": float(np.mean(kps[:,2])),
                        "clf_conf": avg_conf,
                    }
                    self.rep_events.append(event)
                    return True, event, kps_best

        return False, None, kps_best


# 6.5 teller voor hoeveelheid correct/incorrect frames

In [7]:
from pathlib import Path
for cls in ["correct","incorrect"]:
    n = len(list((Path("data/frames/train")/cls).glob("*.jpg")))
    print(cls, n)

correct 295
incorrect 326


# 7. Quick check for my own sanity
Checks whether my trained images are in the correct folders

In [ ]:

model = YOLO(BEST_WEIGHTS)
print("Model names:", model.names)

correct_list = list((Path(OUTPUT_FRAMES_DIR)/"test"/"correct").glob("*.jpg"))
incorrect_list = list((Path(OUTPUT_FRAMES_DIR)/"test"/"incorrect").glob("*.jpg"))
if correct_list and incorrect_list:
    import random
    for img_path in [random.choice(correct_list), random.choice(incorrect_list)]:
        r = model(str(img_path), verbose=False)[0]
    print(
        img_path.name,
        r.names,
        int(r.probs.top1),
        r.names[int(r.probs.top1)],
        float(r.probs.top1conf)
    )
else:
    print("No test images found for quick check.")


Model names: {0: 'correct', 1: 'incorrect'}
Copy of push up 46_f00003.jpg {0: 'correct', 1: 'incorrect'} 0 correct 0.9995589852333069


## 7.5
Quick check for data accuracy and estimations

In [17]:
from ultralytics import YOLO
from pathlib import Path
import random

model = YOLO("runs/pushup-cls42/weights/best.pt")
print(model.names)

for cls in ["correct","incorrect"]:
    p = random.choice(list((Path("data/frames/test")/cls).glob("*.jpg")))
    r = model(str(p), verbose=False)[0]
    pred = model.names[int(r.probs.top1)]
    conf = float(r.probs.top1conf)
    print(f"{p.name}: expected={cls}  predicted={pred}  conf={conf:.2f}")


{0: 'correct', 1: 'incorrect'}
Copy of push up 132_f00004.jpg: expected=correct  predicted=correct  conf=1.00
Copy of push up 46_f00004.jpg: expected=incorrect  predicted=correct  conf=0.99


# 8. Live Push-up Form Test (Webcam) 
Execute the program and run the yolo webcam + rep counter. Feedback is provided shortly on screen, or in the terminal.


In [ ]:
from ultralytics import YOLO
import cv2, time
from pathlib import Path

# load models
RUNS_DIR = "runs"
RUN_NAME = "pushup-cls42"  # dit updaten naar gewenste run
BEST_WEIGHTS = f"{RUNS_DIR}/{RUN_NAME}/weights/best.pt"

pose_model = YOLO("yolov8n-pose.pt")   
clf_model = YOLO(BEST_WEIGHTS)         
print("Classifier loaded:", clf_model.names)

# rep counter init
rep_counter = PoseRepCounter(
    pose_model=pose_model,
    eval_model=clf_model,
    frames_for_classify=7,
    vote_strategy="majority"
)

# webcam setup
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Cannot open webcam")

print("Press 'q' to quit the live test.")

# feedback timer
feedback_text = ""
feedback_end_time = 0

# optional video recorder
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter("pushup_session.mp4", fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # mirror image
    counted, event, kps = rep_counter.update(frame)

    # rep counter on screen
    text = f"Reps: {rep_counter.reps}"
    cv2.putText(frame, text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 2)

    # feedback on screen
    if counted and event:
        feedback = event.get("feedback", "unknown")
        issues = ", ".join(event.get("issues", [])) or "Good form!"
        print(f"Rep {event['rep']}: {feedback.upper()} | {issues}")
        feedback_text = f"{feedback.upper()} - {issues}"
        feedback_end_time = time.time() + 3  # Show for 3 seconds
    
    # display feedback if timer hasn't expired
    if time.time() < feedback_end_time:
        cv2.putText(frame, feedback_text, (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)

    # Display window
    cv2.imshow("Push-up Form Tracker", frame)

    # optional write to output video
    # out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# cleanup
cap.release()
cv2.destroyAllWindows()
# out.release()

Classifier loaded: {0: 'correct', 1: 'incorrect'}
Press 'q' to quit the live test.
Rep 1: INCORRECT | Good form!
Rep 2: INCORRECT | Good form!
Rep 3: INCORRECT | Good form!
Rep 4: INCORRECT | Good form!
Rep 5: INCORRECT | Good form!
Rep 6: INCORRECT | Good form!
Rep 7: INCORRECT | Good form!


C:\Users\timle\AppData\Local\Temp\ipykernel_10160\1309306314.py:16: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  return np.linalg.norm(np.cross(b-a, a-p)) / (np.linalg.norm(b-a) + 1e-6)


Rep 8: INCORRECT | Depth too shallow (elbows not bent enough), Hips sagging (keep core tight)
Rep 9: INCORRECT | Depth too shallow (elbows not bent enough)
Rep 10: INCORRECT | Depth too shallow (elbows not bent enough)
Rep 11: CORRECT | Depth too shallow (elbows not bent enough)
Rep 12: INCORRECT | Depth too shallow (elbows not bent enough)
Rep 13: INCORRECT | Depth too shallow (elbows not bent enough), Hips sagging (keep core tight)
Rep 14: INCORRECT | Good form!
Rep 15: INCORRECT | Depth too shallow (elbows not bent enough)
Rep 16: INCORRECT | Depth too shallow (elbows not bent enough)
Rep 17: INCORRECT | Depth too shallow (elbows not bent enough)
Rep 18: CORRECT | Depth too shallow (elbows not bent enough)
Rep 19: CORRECT | Good form!
Rep 20: CORRECT | Depth too shallow (elbows not bent enough)
Rep 21: CORRECT | Depth too shallow (elbows not bent enough)
Rep 22: CORRECT | Depth too shallow (elbows not bent enough)
Rep 23: CORRECT | Depth too shallow (elbows not bent enough)
Rep 24: 